In [69]:
import pandas as pd
import numpy as np

In [70]:
weather_pd = pd.read_csv('./weather_ts.csv')

In [71]:
weather_pd.head()

,time_stamp,aqi_W San Gabriel Vly,aqi_E San Fernando Vly,aqi_SW Coastal LA,aqi_San Gabriel Mts,aqi_SW San Bernardino,aqi_Southeast LA CO,aqi_South Coastal LA,aqi_Central LA CO,aqi_NW Coastal LA,aqi_Santa Clarita Vly,aqi_W San Fernando Vly,aqi_E San Gabriel V-2
0,2016-11-03 19:00:00,15.0,52.0,22.0,23.0,51.0,33.0,22.0,52.0,22.0,18.0,52.0,23.0
1,2016-11-03 20:00:00,24.0,60.0,32.0,24.0,48.0,40.0,32.0,60.0,32.0,21.0,60.0,24.0
2,2016-11-03 21:00:00,30.0,53.0,43.0,23.0,54.0,56.0,43.0,53.0,43.0,23.0,53.0,18.0
3,2016-11-03 22:00:00,32.0,57.0,46.0,24.0,54.0,64.0,46.0,57.0,46.0,24.0,57.0,18.0
4,2016-11-03 23:00:00,30.0,63.0,40.0,22.0,52.0,57.0,40.0,63.0,40.0,22.0,63.0,9.0


In [72]:
len(weather_pd)

7229

In [73]:
sensors = ['W San Gabriel Vly',
 'E San Fernando Vly',
 'SW Coastal LA',
 'San Gabriel Mts',
 'SW San Bernardino',
 'Southeast LA CO',
 'South Coastal LA',
 'Central LA CO',
 'NW Coastal LA',
 'Santa Clarita Vly',
 'W San Fernando Vly',
 'E San Gabriel V-2']

In [99]:
import numpy as np
import tensorflow as tf


def masked_mse_tf(preds, labels, null_val=np.nan):
    """
    Accuracy with masking.
    :param preds:
    :param labels:
    :param null_val:
    :return:
    """
    if np.isnan(null_val):
        mask = ~tf.is_nan(labels)
    else:
        mask = tf.not_equal(labels, null_val)
    mask = tf.cast(mask, tf.float32)
    mask /= tf.reduce_mean(mask)
    mask = tf.where(tf.is_nan(mask), tf.zeros_like(mask), mask)
    loss = tf.square(tf.subtract(preds, labels))
    loss = loss * mask
    loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
    return tf.reduce_mean(loss)


def masked_mae_tf(preds, labels, null_val=np.nan):
    """
    Accuracy with masking.
    :param preds:
    :param labels:
    :param null_val:
    :return:
    """
    if np.isnan(null_val):
        mask = ~tf.is_nan(labels)
    else:
        mask = tf.not_equal(labels, null_val)
    mask = tf.cast(mask, tf.float32)
    mask /= tf.reduce_mean(mask)
    mask = tf.where(tf.is_nan(mask), tf.zeros_like(mask), mask)
    loss = tf.abs(tf.subtract(preds, labels))
    loss = loss * mask
    loss = tf.where(tf.is_nan(loss), tf.zeros_like(loss), loss)
    return tf.reduce_mean(loss)


def masked_rmse_tf(preds, labels, null_val=np.nan):
    """
    Accuracy with masking.
    :param preds:
    :param labels:
    :param null_val:
    :return:
    """
    return tf.sqrt(masked_mse_tf(preds=preds, labels=labels, null_val=null_val))


def masked_rmse_np(preds, labels, null_val=np.nan):
    return np.sqrt(masked_mse_np(preds=preds, labels=labels, null_val=null_val))


def masked_mse_np(preds, labels, null_val=np.nan):
    with np.errstate(divide='ignore', invalid='ignore'):
        if np.isnan(null_val):
            mask = ~np.isnan(labels)
        else:
            mask = np.not_equal(labels, null_val)
        mask = mask.astype('float32')
        mask /= np.mean(mask)
        rmse = np.square(np.subtract(preds, labels)).astype('float32')
        rmse = np.nan_to_num(rmse * mask)
        return np.mean(rmse)


def masked_mae_np(preds, labels, null_val=np.nan):
    with np.errstate(divide='ignore', invalid='ignore'):
        if np.isnan(null_val):
            mask = ~np.isnan(labels)
        else:
            mask = np.not_equal(labels, null_val)
        mask = mask.astype('float32')
        mask /= np.mean(mask)
        mae = np.abs(np.subtract(preds, labels)).astype('float32')
        mae = np.nan_to_num(mae * mask)
        return np.mean(mae)


def masked_mape_np(preds, labels, null_val=np.nan):
    with np.errstate(divide='ignore', invalid='ignore'):
        if np.isnan(null_val):
            mask = ~np.isnan(labels)
        else:
            mask = np.not_equal(labels, null_val)
        mask = mask.astype('float32')
        mask /= np.mean(mask)
#         mape = np.abs(np.divide(np.subtract(preds, labels).astype('float32'), labels.astype('float32')))
        sub = np.subtract(preds, labels).astype('float32')
        sub = sub * 100
        mape = np.abs(np.divide(sub, labels.astype('float32')))
        mape = np.nan_to_num(mask * mape)
#         print np.mean(mape)
#         mape = 100 * np.array(mape)
#         mape = 100 * mape
        print mape
        return np.mean(mape)


# Builds loss function.
def masked_mse_loss(scaler, null_val):
    def loss(preds, labels):
        if scaler:
            preds = scaler.inverse_transform(preds)
            labels = scaler.inverse_transform(labels)
        return masked_mse_tf(preds=preds, labels=labels, null_val=null_val)

    return loss


def masked_rmse_loss(scaler, null_val):
    def loss(preds, labels):
        if scaler:
            preds = scaler.inverse_transform(preds)
            labels = scaler.inverse_transform(labels)
        return masked_rmse_tf(preds=preds, labels=labels, null_val=null_val)

    return loss


def masked_mae_loss(scaler, null_val):
    def loss(preds, labels):
        if scaler:
            preds = scaler.inverse_transform(preds)
            labels = scaler.inverse_transform(labels)
        mae = masked_mae_tf(preds=preds, labels=labels, null_val=null_val)
        return mae

    return loss


def calculate_metrics(df_pred, df_test, null_val):
    """
    Calculate the MAE, MAPE, RMSE
    :param df_pred:
    :param df_test:
    :param null_val:
    :return:
    """
    mape = masked_mape_np(preds=df_pred.as_matrix(), labels=df_test.as_matrix(), null_val=null_val)
    mae = masked_mae_np(preds=df_pred.as_matrix(), labels=df_test.as_matrix(), null_val=null_val)
    rmse = masked_rmse_np(preds=df_pred.as_matrix(), labels=df_test.as_matrix(), null_val=null_val)
    return mae, mape, rmse

In [75]:
weather_pd = weather_pd.set_index('time_stamp')
weather_train = weather_pd[0:6485]
weather_test = weather_pd[6485:]

In [76]:
weather_check = weather_pd[6484:-1]

In [77]:
len(weather_check)

744

In [78]:
weather_check.as_matrix()

array([[ 52.,  73.,  45., ...,  61.,  73.,  49.],
       [ 47.,  77.,  43., ...,  62.,  77.,  60.],
       [ 53.,  75.,  46., ...,  57.,  75.,  66.],
       ..., 
       [ 49.,  63.,  77., ...,  56.,  63.,  43.],
       [ 61.,  66.,  70., ...,  54.,  66.,  43.],
       [ 70.,  70.,  60., ...,  50.,  70.,  39.]])

In [79]:
weather_test.as_matrix()

array([[ 47.,  77.,  43., ...,  62.,  77.,  60.],
       [ 53.,  75.,  46., ...,  57.,  75.,  66.],
       [ 50.,  76.,  49., ...,  53.,  76.,  60.],
       ..., 
       [ 61.,  66.,  70., ...,  54.,  66.,  43.],
       [ 70.,  70.,  60., ...,  50.,  70.,  39.],
       [ 76.,  68.,  52., ...,  43.,  68.,  25.]])

In [100]:
mae, mape, rmse = calculate_metrics(weather_check, weather_test, np.nan)

[[ 10.63829803   5.19480515   4.65116262 ...,   1.61290324   5.19480515
   18.33333397]
 [ 11.320755     2.66666675   6.52173901 ...,   8.77192974   2.66666675
    9.090909  ]
 [  6.           1.31578946   6.12244892 ...,   7.54716969   1.31578946
   10.        ]
 ..., 
 [ 19.67213058   4.5454545   10.         ...,   3.70370364   4.5454545    0.        ]
 [ 12.85714245   5.71428585  16.66666603 ...,   8.           5.71428585
   10.2564106 ]
 [  7.89473677   2.94117641  15.38461494 ...,  16.2790699    2.94117641
   56.        ]]


In [81]:
def selfMape(df_pred, df_label):
    num = len(df_pred)
    

In [101]:
mae, mape, rmse

(3.9759185, inf, 5.7966104)